In [1]:
import os
import requests
#from elasticsearch import Elasticsearch
from smolagents import CodeAgent, OpenAIServerModel, tool

# --- CONFIGURATION ---
NVIDIA_API_KEY = os.environ.get("NVIDIA_API_KEY")
ELASTIC_API_URL = "http://localhost:9200" # Or your container name "http://elasticsearch:9200"


/home/ysidhom/Documents/M2DS/stream_data/public_transport_RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


# 1. DEFINE THE TOOL
# smolagents uses the docstring and type hints to understand the tool!
@tool
def search_metro_disruptions(user_query: str) -> str:
    """
    Searches for real-time disruption information on the Paris Metro using Vector Search.
    Use this tool when the user asks about traffic, delays, specific lines (like Line 14), or access to airports.
    
    Args:
        user_query: The specific topic or question to search for (e.g. 'Is Line 14 running?', 'Trouble at Orly').
    """
    print(f"🕵️  Tool triggered: Searching for '{user_query}'...")
    
    # A. Generate Embedding (Using NVIDIA API for consistency)
    # We do this manually here so the tool is self-contained
    invoke_url = "https://integrate.api.nvidia.com/v1/embeddings"
    headers = {
        "Authorization": f"Bearer {NVIDIA_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "input": [user_query],
        "model": "nvidia/nv-embedqa-e5-v5",
        "input_type": "query",
        "encoding_format": "float"
    }
    
    try:
        # Get Vector
        resp = requests.post(invoke_url, json=payload, headers=headers, timeout=5)
        resp.raise_for_status()
        query_vector = resp.json()['data'][0]['embedding']
        
        # B. Search Elasticsearch
        es = Elasticsearch(ELASTIC_API_URL, basic_auth=None, meta_header=False)
        
        # Ensure we check the right index
        search_response = es.search(
            index="paris-disruptions",
            knn={
                "field": "embedding_vector",
                "query_vector": query_vector,
                "k": 3,
                "num_candidates": 50
            }
        )
        
        hits = search_response['hits']['hits']
        if not hits:
            return "No active disruptions found for this query. Traffic seems normal."
            
        # C. Format Output
        results = []
        for hit in hits:
            src = hit['_source']
            results.append(f"- [Line {src.get('impact')}] {src.get('title')}: {src.get('description')}")
            
        return "\n".join(results)

    except Exception as e:
        return f"Error during search: {str(e)}"


In [4]:

# 2. CONFIGURE NVIDIA MODEL
# smolagents can connect to NVIDIA NIM using the OpenAIServerModel class
model = OpenAIServerModel(
    model_id="meta/llama-3.1-70b-instruct",
    api_base="https://integrate.api.nvidia.com/v1", # NVIDIA's OpenAI-compatible endpoint
    api_key=NVIDIA_API_KEY
)

# 3. CREATE THE AGENT
# CodeAgent is powerful because it writes Python code to solve the task
agent = CodeAgent(
    tools=[], 
    model=model,
    add_base_tools=True # Adds calculator, etc. if needed
)


In [ ]:
from smolagents import CodeAgent, LiteLLMModel, tool # <--- Changed import

model = LiteLLMModel(
    model_id="openai/meta/llama-3.1-70b-instruct", 
    api_base="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY
)

agent = CodeAgent(
    tools=[], 
    model=model,
    add_base_tools=True
)


🤖 Agent Starting...


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Is there any trouble getting to the airport right now?                                                          │
│                                                                                                                 │
╰─ LiteLLMModel - openai/meta/llama-3.1-70b-instruct ─────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 1: Duration 93.69 seconds]

KeyboardInterrupt: 

In [9]:
import os
import json
from openai import OpenAI
#from tools.retrieve_context import get_disruption_context # Import your RAG tool

# 1. SETUP CLIENT
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ.get("NVIDIA_API_KEY")
)

MODEL_ID = "meta/llama-3.1-70b-instruct"

# 2. DEFINE TOOLS (OpenAI Format)
tools=[]
# tools = [
#     {
#         "type": "function",
#         "function": {
#             "name": "get_disruption_context",
#             "description": "Searches for real-time Paris Metro disruptions.",
#             "parameters": {
#                 "type": "object",
#                 "properties": {
#                     "user_query": {"type": "string", "description": "The search topic"}
#                 },
#                 "required": ["user_query"]
#             }
#         }
#     }
# ]

def run_agent(user_input):
    print(f"🤖 User: {user_input}")
    messages = [{"role": "user", "content": user_input}]

    # First Call
    response = client.chat.completions.create(
        model=MODEL_ID, messages=messages
    )
    
    msg = response.choices[0].message
    
    # Check if tool is called
    if msg.tool_calls:
        print(f"🛠️  Tool Call Detected: {msg.tool_calls[0].function.name}")
        messages.append(msg) # Add 'assistant' message
        
        for tool_call in msg.tool_calls:
            # Execute Python Tool
            args = json.loads(tool_call.function.arguments)
            tool_output = "used tools" #get_disruption_context(args["user_query"])
            
            # Add 'tool' message
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": tool_output
            })

        # Final Response
        final = client.chat.completions.create(model=MODEL_ID, messages=messages)
        return final.choices[0].message.content
    
    return msg.content

if __name__ == "__main__":
    print(run_agent("Is there any trouble on Line 14?"))

🤖 User: Is there any trouble on Line 14?
I'm happy to help, but I'm a text-based AI assistant and do not have real-time access to specific train line information. The information on Line 14 could depend on various factors such as location, time of day, and any scheduled maintenance or disruptions.

However, I can suggest ways for you to find the latest information on Line 14:

1. **Check the official website or app**: You can check the official website or mobile app of the transit agency responsible for Line 14 for the latest updates on service disruptions, maintenance, or planned work.
2. **Social media**: Many transit agencies have social media accounts where they post updates on service disruptions or planned work.
3. **Transit agency's customer service**: You can contact the transit agency's customer service department to ask about the current status of Line 14.
4. **Google search**: A simple Google search with the phrase "Line 14 service updates" or "Line 14 status" can provide yo